# Run an OpenMM simulation with NanoVer 

NanoVer can run OpenMM simulations in two ways: either using [ASE as an interface](../ase/ase_openmm_nanotube.ipynb), or directly using [OpenMM mechanisms](./openmm_nanotube.ipynb). Using the ASE interface offers the most flexibility to customise a workflow; in the [ASE OpenMM graphene example](../ase/ase_openmm_graphene.ipynb) we control the physics parameters of a running simulation from a Jupyter notebook. The ASE interface misses some specific OpenMM features, though; one of the most significant being holonomic constraints. Using the OpenMM mechanisms without ASE as an interface gives access to all of OpenMM features, but may require more work for some customisation needs. Here, we demonstrate how to to use the OpenMM mechanisms with NanoVer.

## Preparing an OpenMM simulation

Running an OpenMM simulation with NanoVer requires two elements: an OpenMM simulation and a NanoVer runner. The simulation is an ordinary OpenMM simulation that we will prepare as normal. We then use this to create an instance of the `OpenMMSimulation` class, which can be passed to NanoVer to run an interactive molecular dynamics (iMD) simulation. An important note is that removing the center of mass motion (as is the default in OpenMM) leads to unintuitive behaviours when you interact with the molecules, particularly for small systems. We recommend deactivating this default flag, i.e. with `removeCMMotion=False`.

Here, we prepare a simulation of a polyalanine following instructions adapted from [the OpenMM documentation](http://docs.openmm.org/7.0.0/userguide/application.html#a-first-example).

In [1]:
import openmm as mm
from openmm import app
from openmm import unit

pdb = app.PDBFile('openmm_files/17-ala.pdb')
forcefield = app.ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1 * unit.nanometer,
    constraints=app.HBonds,
    removeCMMotion=False,
)

integrator = mm.LangevinIntegrator(
    300 * unit.kelvin,
    1 / unit.picosecond,
    0.002 * unit.picoseconds,
)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

Thus far, the simulation is a normal OpenMM simulation and can be used as such. Let's minimize the energy and run a few steps to make sure everything is working.

In [2]:
simulation.minimizeEnergy()
simulation.step(100)

## Running the simulation with NanoVer

NanoVer works using a client-server architecture. A NanoVer runner creates the server and makes a link between that server and the simulation. Here, we create an `OmniRunner` and pass an `OpenMMSimulation` to it. The `OpenMMSimulation` class is responsible for delivering data output by the OpenMM simulation to the client. An `OpenMMSimulation` delivers data in a similar way to a reporter in OpenMM: the data is delivered in frames, which are published at intervals of simulation steps specified by the `frame_interval` property (defaults to 5 steps).

In [3]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

# Create the OpenMMSimulation instance
polyalanine_simulation = OpenMMSimulation.from_simulation(simulation)

# Pass the simulation to the runner
imd_runner = OmniRunner.with_basic_server(polyalanine_simulation, name="polyalanine-omm-server")

# Run the simulation
imd_runner.next()

Excellent! We now have the simulation running and a server waiting for clients to connect!

&nbsp;  

**Note**: Be careful if you run the above cell multiple times without closing the server by running `imd_runner.close()` (see below), as you will start multiple servers, which may be discovered if you use autoconnect. You can guard against this by swapping the final two lines (that define and start the runner, respectively) in the cell above with:


```python
try:
    imd_runner.close()
except NameError: # If the server hasn't been defined yet, there will be an error
    pass
imd_runner = OmniRunner.with_basic_server(polyalanine_simulation, name="polyalanine-omm-server")
imd_runner.next()
```
&nbsp;  

Once you are done, you can close the server to free the network port.

In [4]:
# Close the runner
imd_runner.close()

## Saving a simulation to file

Once you have a simulation ready, you may want to save this setup to a file. By doing so, it becomes simpler to reuse the simulation, including with the command line interface (by running the command `$ nanover-omi --omm my_input_file.xml`).

This can be achieved using `nanover.openmm.serializer.serialize_simulation`, which creates an XML that describes the system, the initial structure, and the integrator.

In [5]:
from nanover.openmm import serializer
with open('polyalanine-simulation.xml', 'w') as outfile:
    outfile.write(serializer.serialize_simulation(simulation))

Once created, the simulation can be retrieved from the file using `nanover.openmm.serializer.deserialize_simulation`, which reads the XML and creates the simulation object.

In [6]:
with open('polyalanine-simulation.xml') as infile:
    simulation_2 = serializer.deserialize_simulation(infile.read())

## Using a saved simulation

With a simulation saved as an XML file, setting up a NanoVer runner becomes much simpler. Below we set up an `OpenMMSimulation` for polyalanine by retrieving the simulation directly from a pre-prepared NanoVer OpenMM XML file, but you could do the same with the file we created in this tutorial by commenting out the top line and uncommenting the second line:

In [7]:
polyalanine_simulation = OpenMMSimulation.from_xml_path('openmm_files/17-ala.xml')
# polyalanine_simulation = OpenMMSimulation.from_xml_path('polyalanine-simulation.xml')

Now the simulation can be run in the same fashion as demonstrated previously.

In [8]:
imd_runner = OmniRunner.with_basic_server(polyalanine_simulation, name="polyalanine-omm-server")
imd_runner.next()

Again, once we're finished we should be careful to clean up after ourselves properly by closing the runner as below.

In [9]:
imd_runner.close()

## Saving the trajectory

Although the primary benefit of iMD-VR is the ability to simultaneously visualise and interact with molecular systems, it is often useful to save the trajectory produced in order to run analyses after the simulation is complete.

### Via OpenMM

Saving the trajectory can be done in the regular way for OpenMM simulations: by attaching a reporter. Here we attach a DCD reporter to save the trajectory in the DCD format every 50 frames.

**Note**: It is essential to load the simulation _before_ attaching a DCD reporter, using the `.load()` function.

In [10]:
# Load the simulation
polyalanine_simulation.load()

# Attach the DCD reporter
dcd_reporter = app.DCDReporter('output.dcd', 50)
polyalanine_simulation.simulation.reporters.append(dcd_reporter)

# Run the simulation
imd_runner = OmniRunner.with_basic_server(polyalanine_simulation, name="polyalanine-omm-server")
imd_runner.next()

As ever, let's make sure we close the runner once we're finished with our simulation by running the cell below.

In [11]:
imd_runner.close()

The DCD reporter does not close the file when the simulation is finished. In some cases, this can prevent to open the trajectory with an other software as long as the jupyter kernel is running. This line closes the file. Note that this will break the reporter in case you want to continue running the simulation.

In [13]:
# Tell the DCDReporter to clean itself up (close the DCD file)
dcd_reporter.__del__()

### Via NanoVer

Alternatively, the trajectory of the simulation can be saved using the in-built recording functionality of NanoVer. This allows us to save the simulation output and load the simulation back into NanoVer directly to visualise the recorded trajectory (note that this can also be done with a DCD trajectory using [MDAnalysis with NanoVer](../mdanalysis/mdanalysis_trajectory.ipynb)). In addition, this functionality allows you to play back the shared state (synchronised with the trajectory), enabling you to e.g. visualise the avatars interacting with the system in VR.

When recording the trajectory this way, we create two files: 
* a trajectory file (with the `.traj` extension) that contains the information from the simulation (i.e. the simulation [frames](../fundamentals/frame.ipynb))
* a state file (with the `.state` extension) that contains the updates to the [shared state](../fundamentals/commands_and_state.ipynb)

More information about the contents of these files can be found on the [Recording data](https://irl2.github.io/nanover-docs/concepts/recording.html) page of the NanoVer documentation, but at this stage these details are unimportant. The bottom line is that recording both of these data streams allows NanoVer to replay the recorded trajectory on a server. 

In order to record our simulation, we need to import the `nanover.omni.record.record_from_server` function:


In [14]:
from nanover.omni.record import record_from_server

Next, we need to define the paths of the trajectory and state files that we want to write to.

In [15]:
traj_path = 'polyalanine_recording.traj'
state_path = 'polyalanine_recording.state'

Now we can set up our simulation and server as before, passing the `OmniRunner` to the `record_from_server` function **before** we start the simulation. We'll use the pre-prepared polyalanine NanoVer OpenMM XML again to define our system.

In [16]:
polyalanine_simulation = OpenMMSimulation.from_xml_path('openmm_files/17-ala.xml')
imd_runner = OmniRunner.with_basic_server(polyalanine_simulation, name="polyalanine-omm-server")

# Pass the runner to the recording function
record_from_server(f"localhost:{imd_runner.app_server.port}", traj_path, state_path)

# Start the simulation
imd_runner.next()

Once we're finished with the simulation, we terminate the runner as usual.

In [17]:
imd_runner.close()

Now we have our recorded simulation! 

## Replaying a recorded NanoVer simulation


Having recorded a simulation using NanoVer's in-built functionality for recording, the recording can be passed to an `OmniRunner`, which can serve the recorded simulation, enabling multiple clients to connect to the server to visualise it. In order to do this, we must first import the `PlaybackSimulation` class:

In [18]:
from nanover.omni.playback import PlaybackSimulation

Now we can create an instance of the `PlaybackSimulation` class from the trajectory and state files created in the previous section for our polyalanine simulation, and serve it using an `OmniRunner`. The process is very similar to setting up a regular NanoVer OpenMM simulation from an XML file.

In [19]:
# Create the PlaybackSimulation object
polyalanine_recording = PlaybackSimulation(name="polyalanine-recording",
                                           traj=traj_path,
                                           state=state_path)

# Pass the PlaybackSimulation to a runner and play the recording
recording_runner = OmniRunner.with_basic_server(polyalanine_recording, name="polyalanine-recording-server")
recording_runner.next()

And that's it! Now the recording is up and running on a NanoVer server, and can be visualised by connecting a client to this server.

As ever, once we are finished with the server playing the recording, we should terminate the runner in the usual fashion.

In [20]:
recording_runner.close()

## Next steps

* Check out our [nanotube tutorial](openmm_nanotube.ipynb) to learn how to interact with an OpenMM simulation via a python client
* Learn how to change the visual representation of a [protein-ligand system](openmm_neuraminidase.ipynb)
* Learn how trajectories recorded directly with NanoVer can be [analysed using MDAnalysis](../mdanalysis/mdanalysis_nanover_recording.ipynb)